In [5]:
import time
import random

In [6]:
import pandas as pd
import numpy as np

In [1]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=4, n_workers=1)
client

Client Scheduler: tcp://127.0.0.1:50343 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 4 Memory: 17.05 GB


In [2]:
def costly_simulation(list_param):
    time.sleep(random.random())
    return sum(list_param)

In [3]:
%time costly_simulation([1, 2, 3, 4])

Wall time: 87.8 ms


10

In [7]:
input_params = pd.DataFrame(
    np.random.random(size=(500, 4)),
    columns=['param_a', 'param_b', 'param_c', 'param_d']
)
input_params.head()

,param_a,param_b,param_c,param_d
0,0.988009,0.519150,0.312808,0.753648
1,0.644451,0.239635,0.374190,0.498817
2,0.382101,0.562248,0.620251,0.725072
3,0.098177,0.274796,0.323242,0.471218
4,0.848749,0.688376,0.578596,0.607416


In [8]:
futures = []
for parameters in input_params.values:
    future = client.submit(costly_simulation, parameters)
    futures.append(future)

In [9]:
results = client.gather(futures)

In [10]:
results[:5]

[2.5736146082382207,
 1.757093297785722,
 2.2896714120471655,
 1.1674334122060324,
 2.7231370637369463]

In [12]:
futures = client.map(costly_simulation, input_params.values)

In [14]:
results = client.gather(futures)
len(results)

500